In [6]:
# disperse
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx
import requests

secret = 61740721216174072121
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
target_amount = 0.1
fee = 0.005

response = requests.get('https://testnet.blockexplorer.com/api/addr/{}/utxo'.format(my_address))
utxos = response.json()

student_addresses = [
]
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['amount']
    # create a new tx input
    tx_ins.append(TxIn(
            prev_tx=prev_tx,
            prev_index=prev_index,
            script_sig=b'',
            sequence=0xffffffff,
        ))

# initialize outputs
tx_outs = []

for target_address in student_addresses:
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = int(target_amount*100000000)
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_satoshis,
        script_pubkey=script_pubkey,
    ))

change_amount = total - target_amount*len(student_addresses) - fee

h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
change_satoshis = int(change_amount*100000000)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv, SIGHASH_ALL)

# SANITY CHECK: fee is reasonable
if tx_obj.fee(testnet=True) > 0.05*100000000 or tx_obj.fee(testnet=True) <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

010000001ac56585e8bed772fcc6af3d781fb4f0f0c0d9a9a79d854f135358d48f3fc7da1e000000006a47304402203b7ef36d28e1d7946626835244ab806e9d592ecd8c37a1acb4121adf5b526a00022057dc489d3a37574c92c102f616d6fa142d326e444c9e89a63d80930bf3f7e5ec012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55beffffffff71db5d28a5b9a517ae6f9e86512925027274828f0db446a4b86aeb7360f71a61000000006a47304402207f6ccb63fe85577daf575ba0acb78e56b8b68a93e881e645c93c9c38b03cfef502201606b6e390fa8b4a6e7818bb04a019b5c8e589099b73940a7868b9842e753d17012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55beffffffff5b0e3745bc15acf17d59327610e0a25a22930df1e64a78312005d3211516fe55000000006b483045022100ba6c3af05133bc6213e7a99b21db534328cc66b813960f4684781025b5a4628a02206a67597f2dd0bc553de00021600f1fc4818581224b6367365ca416dcfbaf1785012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55beffffffffb27561f099e2de41edcc12cd4853c60aaa44ce7b22ab9d9549d9ceacf235c6e3000000006a4730440220008b182e5b46a92462b18d